In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import sys

In [ ]:
%%bash
sudo apt-get install dnsutils 
myip="$(dig +short myip.opendns.com @resolver1.opendns.com)"
echo "My WAN/Public IP address: ${myip}"
echo $myip > myip.txt

In [ ]:
#The following line is only needed for deployment. Don't run it. If you do maybe restart your kernel
os.chdir(os.path.dirname(sys.argv[0]))

In [ ]:
isMaster=1;
verbose=0;

Create template bash script. This script will query your ip, save it to myip.txt and run the python module to update your google drive folder:

In [ ]:
%%bash
cat >fetchAndUpdateIP.sh <<EOL
#!/bin/bash
myip="$(dig +short myip.opendns.com @resolver1.opendns.com)"
echo "My WAN/Public IP address: ${myip}"
echo $myip > myip.txt
python3 OUTPUT_PATH/UpdateIpOnGoogleDrive.py
EOL


In [ ]:
ipFile='myip.txt'

## The following cell will access the google drive via the api 
## When running it for the first time, it will access your drive and ask you to verify the access, which has to be done from a browser. After that it will store the access key and your credentials. Then the module will be able to run from anywhere with the access and token file.  

In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.file']
# Run this script in mastermode
def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
     # Call the Drive v3 API
    service = build('drive', 'v3', credentials=creds)
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name, trashed)").execute()
    items = results.get('files', [])
    if not items and verbose:
        print('No files found.')
    else:
        if verbose:
            print('Files:')
        for item in items:
            print(u'{0} ({1}), {2}'.format(item['name'], item['id'],item['trashed'])) 
            ##Make sure to delete only the old file. If you have other files with the same name they will be deleted, too
            if isMaster and not item['trashed'] and item['name']==ipFile:
                service.files().delete(fileId=item['id']).execute()
            FILES=((ipFile,1),item['id'])  # 1 = Let google drive try to automatically convert your document
            file_metadata = {'name': ipFile}
            media = MediaFileUpload(ipFile,          ) # you can add your mime type here ~ See google drive api docs 
            file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()
            print ('File ID: %s' % file.get('id'))
    return items
if __name__ == '__main__':
    aa=main()

# NOW we deploy the appication to the target in the network


## 1. Convert this notebook to a python file

In [ ]:
!jupyter nbconvert --to python 'UpdateIpOnGoogleDrive.ipynb' --template=pyCode.tpl


In [ ]:
%%bash 
pipreqs --force ./

## 2. Setup deploy template script 
Here you can change the update interval by changing this line: '*/45 * * * * sh OUTPUT_PATH/fetchAndUpdateIP.sh'.
See 'man crontab' for details

In [36]:
%%bash
cat >deploy.sh <<EOL
#!/bin/bash
scp ./fetchAndUpdateIP.sh myPieUname@myPieIp:OUTPUT_PATH
scp ./UpdateIpOnGoogleDrive.py myPieUname@myPieIp:OUTPUT_PATH
scp ./requirements.txt myPieUname@myPieIp:OUTPUT_PATH
scp ./credentials.json myPieUname@myPieIp:OUTPUT_PATH
scp ./token.pickle myPieUname@myPieIp:OUTPUT_PATH
ssh myPieUname@myPieIp "crontab -l > OUTPUT_PATH/tmp.txt & \
echo '*/45 * * * * sh OUTPUT_PATH/fetchAndUpdateIP.sh' | tee -a OUTPUT_PATH/tmp.txt & crontab -u pi OUTPUT_PATH/tmp.txt & \
sudo apt-get install dnsutils & \
python3 -m pip install -r OUTPUT_PATH/requirements.txt"
EOL

## 3. Find out your targets ip address. SSH needs to be enabled with your public ssh-key on the remote machine. FYI - https://www.ssh.com/ssh/copy-id 

In [ ]:
%%bash
outPath='\/home\/pi\/Public' # backslashes are needed to escape the forward slashes in sed. See "man sed"
targetIp="192.168.1.51"
remoteUser='pi'
sed -i "s/OUTPUT_PATH/${outPath}/g" fetchAndUpdateIP.sh

targetIp="192.168.1.51"
remoteUser='pi'
sed -i "s/myPieIp/${targetIp}/g" deploy.sh
sed -i "s/myPieUname/${remoteUser}/g" deploy.sh
sed -i "s/OUTPUT_PATH/${outPath}/g" deploy.sh


## Finally run scripts:

In [ ]:
sh fetchAndUpdateIP.sh
sh deploy.sh
